# Detectron 2 Training

## Installing Dependences

we will be using `pip` to install dependencies.
Note: use cu101 as colab has CUDA 10.1

### Pytorch

In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab

### Detectron

Installing Detectron and dependences

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

##  Google Drive


Mouting your Google drive for dataset and model saving.

This even though not a required step but it is recommended so as to store the model once trained

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Model Creation and Training

### Import Modules

Here we import all the modules and packages needed for
- importing the dataset
- processing the dataset
- creating the model
- training the model

In [ ]:

import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

### Dataset

In [ ]:
!wget https://storage.googleapis.com/bucket-8732/car_damage/preprocessed.zip
!unzip preprocessed.zip

Once we have downloaded the dataset we need to regester it as coco format for training

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("train", {}, "/content/outputs/output.json", "/content/preprocessed/image/")


In [ ]:

register_coco_instances("train1", {}, "/content/train.json", "/content/trainimages/")


Saving the Metadata of the dataset

In [ ]:
sample_metadata = MetadataCatalog.get("train")
dataset_dicts = DatasetCatalog.get("train")
sample_metadata1 = MetadataCatalog.get("train1")
dataset_dicts1 = DatasetCatalog.get("train1")

### Dataset Exploration (Optional)

Exploring the Data set before traning.

In [ ]:
import random

for d in random.sample(dataset_dicts1, 4):
    file_name = d["file_name"]
    print(file_name)
    img = cv2.imread(file_name)
    visualizer = Visualizer(img[:, :, ::-1], metadata=sample_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

### Model Creation


Here we add the important options to the model configuration object that we will later use for training the model.

The basic model hyperparameters and parameters are set here.

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
cfg.merge_from_file("./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN = ("train","train1")
cfg.DATASETS.TEST = ()   # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.CLIP_GRADIENTS.ENABLED=True
cfg.INPUT.CROP.ENABLED = True
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.005
cfg.SOLVER.MAX_ITER = 25000  
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # 3 classes (Person, Helmet, Car)
cfg.OUTPUT_DIR = "/content/outputs"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

### Model Training

In [ ]:
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()